In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df_sup = pd.read_excel('df_surprise_labeled.xlsx')
print(len(df_sup))

600


In [3]:
for cl in df_sup.columns:
    if(cl=='Text'):
        continue
    df_sup[cl].loc[(df_sup[cl]>0)]=1
    print(df_sup[cl].value_counts())

0    331
1    269
Name: Joy, dtype: int64
0    457
1    143
Name: Sadness, dtype: int64
0    541
1     59
Name: Anger, dtype: int64
0    585
1     15
Name: Disgust, dtype: int64
0    551
1     49
Name: Admiration, dtype: int64
1    567
0     33
Name: Surprise, dtype: int64
0    374
1    226
Name: Interest, dtype: int64
0    577
1     23
Name: Fear, dtype: int64


C:\Users\Rainmaker\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\Rainmaker\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\Rainmaker\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [4]:
for cl in df_sup.columns:
    if(cl=='Text'):
        continue
    print(df_sup[cl].value_counts())

0    331
1    269
Name: Joy, dtype: int64
0    457
1    143
Name: Sadness, dtype: int64
0    541
1     59
Name: Anger, dtype: int64
0    585
1     15
Name: Disgust, dtype: int64
0    551
1     49
Name: Admiration, dtype: int64
1    567
0     33
Name: Surprise, dtype: int64
0    374
1    226
Name: Interest, dtype: int64
0    577
1     23
Name: Fear, dtype: int64


In [5]:
df = pd.read_excel('hand_7k.xlsx')
print(len(df))

7101


In [6]:
df = df.append(df_sup)
print(len(df))

7701


In [7]:
df = df.sample(7701).reset_index(drop=True)
print(len(df))

7701


In [8]:
for cl in df.columns:
    if(cl=='Text'):
        continue
    print(df[cl].value_counts())

0    5794
1    1907
Name: Joy, dtype: int64
0    4026
1    3675
Name: Sadness, dtype: int64
0    6857
1     844
Name: Anger, dtype: int64
0    6381
1    1320
Name: Disgust, dtype: int64
0    7142
1     559
Name: Admiration, dtype: int64
0    6900
1     801
Name: Surprise, dtype: int64
0    6048
1    1653
Name: Interest, dtype: int64
0    7089
1     612
Name: Fear, dtype: int64


In [9]:
singleword=[]
for key,val in senticnet.items():
    if(len(key.split('_'))==1):
        singleword.append(key)
print(len(singleword))
word=[]
primary=[]
sec=[]
pola=[]
for x in singleword:
    word.append(x)
    primary.append(senticnet[x][4])
    sec.append(senticnet[x][5])
    pola.append(senticnet[x][7])
df_emo=pd.DataFrame(list(zip(word,primary,sec,pola)),columns=["Word","Primary","Secondary","Polarity"])
df_emo.sample(10)

39891


,Word,Primary,Secondary,Polarity
12590,faker,#sadness,#disgust,-0.82
4393,brutality,#anger,#disgust,-0.96
19380,killjoy,#sadness,#disgust,-0.75
20880,lyricist,#joy,#joy,0.937
21930,methanol,#sadness,#anger,-0.72
34702,swab,#joy,#admiration,0.756
35546,thirteen-year-old,#anger,#disgust,-0.84
703,afrocentrist,#sadness,#disgust,-0.89
2122,aspd,#surprise,#admiration,0.947
24198,objectivity,#joy,#admiration,0.899


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [11]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Text'])
print(len(vectorizer.vocabulary_))

15235


In [12]:
df['Sadness'].value_counts()

0    4026
1    3675
Name: Sadness, dtype: int64

In [13]:
df_joy = df[df['Joy']==1]
tmp = df[df['Joy']==0].sample(len(df_joy))
df_joy = df_joy.append(tmp).reset_index(drop=True)
df_joy['Joy'].value_counts()

1    1907
0    1907
Name: Joy, dtype: int64

In [14]:
tag = []
total_len = []
train_cnt = []
test_cnt = []
accu = []
conf_mat = []
for cl in df.columns:
    if(cl=='Text'):
        continue
    df_joy = df[df[cl]==1]
    print(cl)
    tag.append(cl)
    tmp = df[df[cl]==0].sample(len(df_joy))
    df_joy = df_joy.append(tmp).reset_index(drop=True)
    df_joy = df_joy.sample(len(df_joy)).reset_index(drop=True)
    print(len(df_joy))
    total_len.append(len(df_joy))
    X = vectorizer.transform(df_joy['Text'])
    Y = df_joy[cl]
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state = 42)
    train_cnt.append(len(y_train))
    test_cnt.append(len(y_test))
    rdforest = RandomForestClassifier(n_estimators=300)
    rdforest.fit(x_train,y_train)
    y_pred = rdforest.predict(x_test)
    print(accuracy_score(y_test,y_pred))
    accu.append(accuracy_score(y_test,y_pred))
    conf_mat.append(confusion_matrix(y_test,y_pred))
    #classification_report.append(classification_report(y_test,y_pred))

Joy
3814
0.7085953878406709
Sadness
7350
0.6653971708378672
Anger
1688
0.6398104265402843
Disgust
2640
0.5848484848484848
Admiration
1118
0.6178571428571429
Surprise
1602
0.7780548628428927
Interest
3306
0.6614268440145102
Fear
1224
0.6993464052287581


In [15]:
df_without_analysis = pd.DataFrame(list(zip(tag,total_len,train_cnt,test_cnt,accu)),
                                   columns = ['Emotion','Total','Train','Test','Accuracy'])
print(df_without_analysis)

      Emotion  Total  Train  Test  Accuracy
0         Joy   3814   2860   954  0.708595
1     Sadness   7350   5512  1838  0.665397
2       Anger   1688   1266   422  0.639810
3     Disgust   2640   1980   660  0.584848
4  Admiration   1118    838   280  0.617857
5    Surprise   1602   1201   401  0.778055
6    Interest   3306   2479   827  0.661427
7        Fear   1224    918   306  0.699346


In [16]:
df_sum = df.drop('Text',axis=1)
print(len(df_sum))

7701


In [17]:
df_sum_cnt = df_sum.sum(axis=1)
print(df_sum_cnt.value_counts())

1    4056
2    2764
3     498
0     312
4      62
5       9
dtype: int64


In [18]:
def remove_count_user_mentions(tweet):
    tweet_mentions_removed = re.subn(r'@[A-Za-z0-9]+','',tweet)
    tweet = tweet_mentions_removed[0]
    no_user_mentions = tweet_mentions_removed[1]
    return tweet,no_user_mentions
#%%
def remove_count_urls(tweet):
    tweet_url_removed = re.subn('https?://[A-Za-z0-9./]+','',tweet)
    tweet = tweet_url_removed[0]
    no_urls = tweet_url_removed[1]
    return tweet,no_urls
#%%
def remove_count_hashtags(tweet):
    no_hashtags = len({tag.strip("#") for tag in tweet.split() if tag.startswith("#")})
    tweet = re.sub("[^a-zA-Z]", " ",tweet)
    return tweet,no_hashtags    
def get_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [19]:
need = ["J","N","V","R"]
#need = ["V"]
neg = ["n't","not"]
punct = [".",",","?",";","!"]
opposite = {}
opposite["#joy"] = "#sadness"
opposite["#sadness"] = "#joy"
opposite["#admiration"] = "#anger"
opposite["#anger"] = "#admiration"
opposite["#surprise"] = "#fear"
opposite["#fear"] = "#surprise"
opposite["#interest"] = "#disgust"
opposite["#disgust"] = "#interest"

In [20]:
def normal_algo(sen):
    sen = sen.lower()
    sen,removed_user_cnt = remove_count_user_mentions(sen)
    sen,removed_url_cnt = remove_count_urls(sen)
    sen,removed_hashtag_cnt = remove_count_hashtags(sen)
    #print(sen)
    tokens = word_tokenize(sen)
    lem = [wordnet_lemmatizer.lemmatize(t,get_pos(t)) for t in tokens]
    #print(lem)
    lem_lookup = {}
    for i in range(len(tokens)):
        lem_lookup[tokens[i]]=lem[i]
    mark_neg = {}
    nflag = False
    for t in lem:
        if(nflag==True):
            mark_neg[t]=1
        if(t in neg):
            nflag=True
        if(t[0] in punct):
            nflag=False
    tag1 = pos_tag(tokens)
    #print(tag1)
    tokens.clear()
    for x in tag1:
        #print(x)
        if(x[1][0] in need):
            tokens.append(x[0])
    val = {}
    #print(tokens)
    ret_str = ""
    for t in tokens:
        t=lem_lookup[t]
        ret_str+=t
        ret_str+=" "
        if(t in senticnet):
            x = senticnet[t][4]
            #print(t)
            if(t in mark_neg):
                #print(t)
                x=opposite[x]
                #print(t,x)
            if(x in val):
                val[x]+=1
            else:
                val[x]=1
    return ret_str

In [21]:
normal_algo("I feared for my life")

'i fear life '

In [22]:
qmark = []
exmark = []
f=0
for txt in df['Text']:
    f=1
    for lt in txt:
        if(lt=='?'):
            qmark.append(1)
            f=0
            break
    if(f==1):
        qmark.append(0)
    f=1
    for lt in txt:
        if(lt=='!'):
            exmark.append(1)
            f=0
            break
    if(f==1):
        exmark.append(0)
print(len(qmark))
print(len(exmark))

7701
7701


In [23]:
print(pd.Series(qmark).value_counts())
print(pd.Series(exmark).value_counts())

0    6763
1     938
dtype: int64
0    5533
1    2168
dtype: int64


In [24]:
analysed = [normal_algo(txt) for txt in df['Text']]
print(analysed[0:10])

['lol i thought be tucked up bed thanks info have think get back n ', 'need new job please help ', 'pep project be kick butt ', 'look job eb declare bankruptcy today ', 'up twitter world i need help boy problem act so sweet friend s jerk i need boy help ', 'aaaah i need stop sleep late i dread work be i m sleepies ', 'drink hot coffee shirt be prepared burn chest ', 'yeah do i have say i felt so bad um twitter ate rainbow paddle pop xx ', 'commercial shoot be cancel next week too gray nice shot building dress nice nothing jk cool ', 'flood message phone phone be still ill not now please be howl pain ']


In [25]:
df['Analysed'] = analysed
df['qmark'] = qmark
df['exmark'] = exmark
df.head(10)

,Text,Joy,Sadness,Anger,Disgust,Admiration,Surprise,Interest,Fear,Analysed,qmark,exmark
0,@daisyclover1938 lol... I thought you were tuc...,0,0,0,0,0,0,1,0,lol i thought be tucked up bed thanks info hav...,1,1
1,needs a new job. please help,1,0,0,0,0,0,0,0,need new job please help,0,0
2,this pep8 project is kicking our butts.,0,0,1,1,0,0,0,0,pep project be kick butt,0,0
3,"looking for jobs, EB declared bankruptcy today",0,1,0,0,0,0,1,0,look job eb declare bankruptcy today,0,0
4,What up twitter world .I need help boy problem...,1,0,0,0,1,1,0,0,up twitter world i need help boy problem act s...,0,1
5,Aaaah! I need to stop sleeping late! I dread w...,0,1,0,0,0,0,0,1,aaaah i need stop sleep late i dread work be i...,0,1
6,If you drink hot coffee with your shirt off......,0,0,0,0,0,0,0,1,drink hot coffee shirt be prepared burn chest,0,0
7,@charlii1 yeah it did and I had to say no to a...,0,1,0,0,0,0,0,0,yeah do i have say i felt so bad um twitter at...,0,1
8,commercial shoot was cancelled until next week...,1,0,1,0,0,0,0,0,commercial shoot be cancel next week too gray ...,0,1
9,Another flood of messages in my phone. my pho...,0,1,0,0,0,0,0,1,flood message phone phone be still ill not now...,1,0


In [26]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Analysed'])
print(len(vectorizer.vocabulary_))

9606


In [27]:
def get_senticnet(word,em):
    em = '#'+em.lower()
    if(senticnet[word][4]==em or senticnet[word][5]==em):
        return 1
    else:
        return 0

In [28]:
cl = 'Joy'
df_joy = df[df[cl]==1]
X = df_joy[['Analysed','qmark','exmark']]
X.shape

(1907, 3)

In [29]:
cl = 'Joy'
df_joy = df[df[cl]==1]
#print(cl)
#tag.append(cl)
tmp = df[df[cl]==0].sample(len(df_joy))
df_joy = df_joy.append(tmp).reset_index(drop=True)
df_joy = df_joy.sample(len(df_joy)).reset_index(drop=True)
print(len(df_joy))
total_len.append(len(df_joy))
#X = vectorizer.transform(df_joy['Analysed'])
X = df_joy[['Analysed','qmark','exmark']]
Y = df_joy[cl].tolist()
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state = 42)
x_train_analysed = x_train['Analysed'].tolist()
x_train_qmark = x_train['qmark'].tolist()
x_train_exmark = x_train['exmark'].tolist()
x_test_analysed = x_test['Analysed'].tolist()
x_test_qmark = x_test['qmark'].tolist()
x_test_exmark = x_test['exmark'].tolist()
pre = {}
for sen in x_train_analysed:
    tok = word_tokenize(sen)
    for t in tok:
        pre[t]=1
for sen in x_test_analysed:
    tok = word_tokenize(sen)
    for t in tok:
        if(t in pre):
            continue
        else:
            if(t in senticnet):
                x_train_analysed.append(t)
                x_train_qmark.append(0)
                x_train_exmark.append(0)
                y_train.append(get_senticnet(t,cl))
x_train_analysed = vectorizer.transform(x_train_analysed)
x_test_analysed = vectorizer.transform(x_test_analysed)
tmp = sparse.hstack((x_train_analysed,np.array(x_train_qmark)[:,None]))
x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
print(x_train.shape)
tmp = sparse.hstack((x_test_analysed,np.array(x_test_qmark)[:,None]))
x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
print(x_test.shape)
rdforest = RandomForestClassifier(n_estimators=300)
rdforest.fit(x_train,y_train)
y_pred = rdforest.predict(x_test)
print(accuracy_score(y_test,y_pred))

3814
(3350, 9608)
(954, 9608)
0.7064989517819706


In [30]:
tag = []
total_len = []
train_cnt = []
test_cnt = []
accu = []
conf_mat = []
for cl in df.columns:
    if(cl=='Text' or cl=='Analysed'):
        continue
    df_joy = df[df[cl]==1]
    print(cl)
    tag.append(cl)
    tmp = df[df[cl]==0].sample(len(df_joy))
    df_joy = df_joy.append(tmp).reset_index(drop=True)
    df_joy = df_joy.sample(len(df_joy)).reset_index(drop=True)
    print(len(df_joy))
    total_len.append(len(df_joy))
    #X = vectorizer.transform(df_joy['Analysed'])
    X = df_joy[['Analysed','qmark','exmark']]
    Y = df_joy[cl].tolist()
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state = 42)
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    y_train.append(get_senticnet(t,cl))
    x_train_analysed = vectorizer.transform(x_train_analysed)
    x_test_analysed = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    #print(x_train.shape)
    tmp = sparse.hstack((x_test_analysed,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    #print(x_test.shape)
    rdforest = RandomForestClassifier(n_estimators=300)
    rdforest.fit(x_train,y_train)
    y_pred = rdforest.predict(x_test)
    print(accuracy_score(y_test,y_pred))
    accu.append(accuracy_score(y_test,y_pred))
    #conf_mat.append(confusion_matrix(y_test,y_pred))
    #classification_report.append(classification_report(y_test,y_pred))

Joy
3814
0.7012578616352201
Sadness
7350
0.6632208922742111
Anger
1688
0.6208530805687204
Disgust
2640
0.6348484848484849
Admiration
1118
0.575
Surprise
1602
0.7855361596009975
Interest
3306
0.6928657799274486
Fear
1224
0.7745098039215687
qmark
1876
0.997867803837953
exmark
4336
1.0


In [31]:
df_with_analysis = pd.DataFrame(list(zip(tag,total_len,train_cnt,test_cnt,accu)),
                                   columns = ['Emotion','Total','Train','Test','Accuracy'])
print(df_with_analysis)
print(df_without_analysis)

Empty DataFrame
Columns: [Emotion, Total, Train, Test, Accuracy]
Index: []
      Emotion  Total  Train  Test  Accuracy
0         Joy   3814   2860   954  0.708595
1     Sadness   7350   5512  1838  0.665397
2       Anger   1688   1266   422  0.639810
3     Disgust   2640   1980   660  0.584848
4  Admiration   1118    838   280  0.617857
5    Surprise   1602   1201   401  0.778055
6    Interest   3306   2479   827  0.661427
7        Fear   1224    918   306  0.699346


In [32]:
df_test = pd.DataFrame()
for cl in df.columns:
    if(cl=="Text" or cl=="Analysed" or cl=='qmark' or cl=='exmark'):
        continue
    df_test = df_test.append(df[df[cl]==1].sample(100))
    print(len(df_test))

100
200
300
400
500
600
700
800


In [33]:
for cl in df_test.columns:
    if(cl=="Text" or cl=="Analysed"):
        continue
    print(df_test[cl].value_counts())

0    560
1    240
Name: Joy, dtype: int64
0    493
1    307
Name: Sadness, dtype: int64
0    631
1    169
Name: Anger, dtype: int64
0    628
1    172
Name: Disgust, dtype: int64
0    667
1    133
Name: Admiration, dtype: int64
0    637
1    163
Name: Surprise, dtype: int64
0    586
1    214
Name: Interest, dtype: int64
0    671
1    129
Name: Fear, dtype: int64
0    690
1    110
Name: qmark, dtype: int64
0    558
1    242
Name: exmark, dtype: int64


In [34]:
print(len(df_test))

800


In [35]:
df_test.head()

,Text,Joy,Sadness,Anger,Disgust,Admiration,Surprise,Interest,Fear,Analysed,qmark,exmark
3233,@Area224 I can relate...I had a guard because ...,1,0,0,0,0,0,0,0,i relate i have guard i clench jaw still do i ...,0,1
2846,"@DarthSLR Time to make lemonaide, Nik!",1,0,0,0,0,0,0,0,time make lemonaide nik,0,1
2721,hahaha math exam was shit,1,0,0,0,0,0,0,0,hahaha math exam be shit,0,0
3812,"@GeemaPee Just been to asda, now at the fort w...",1,0,0,0,0,0,0,0,just be asda now fort wait hour photo everythi...,0,1
5417,@fyldelibraries i am looking forward to QUEST ...,1,0,0,0,0,1,0,0,i be look forward quest seeker woohoo shame ca...,0,1


In [36]:
df = df[~df.index.isin(df_test.index)]
df_test = df_test[~df_test.index.duplicated(keep='first')].reset_index(drop=True)
df = df.reset_index(drop=True)
print(len(df))
print(len(df_test))

6919
782


In [37]:
g_test_analysed = vectorizer.transform(df_test['Analysed'])
g_test_qmark = df_test['qmark'].tolist()
g_test_exmark = df_test['exmark'].tolist()
tmp = sparse.hstack((g_test_analysed,np.array(g_test_qmark)[:,None]))
g_test = sparse.hstack((tmp,np.array(g_test_exmark)[:,None]))

In [38]:
tag = []
total_len = []
train_cnt = []
test_cnt = []
accu = []
conf_mat = []
predict_score_list = []
for cl in df.columns:
    if(cl=='Text' or cl=='Analysed' or cl=='qmark' or cl=='exmark'):
        continue
    df_joy = df[df[cl]==1]
    print(cl)
    tag.append(cl)
    tmp = df[df[cl]==0].sample(len(df_joy))
    df_joy = df_joy.append(tmp).reset_index(drop=True)
    df_joy = df_joy.sample(len(df_joy)).reset_index(drop=True)
    print(len(df_joy))
    total_len.append(len(df_joy))
    #X = vectorizer.transform(df_joy['Analysed'])
    X = df_joy[['Analysed','qmark','exmark']]
    Y = df_joy[cl].tolist()
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    add_cnt = 0
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    add_cnt+=1
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    y_train.append(get_senticnet(t,cl))
    x_train_analysed = vectorizer.transform(x_train_analysed)
    x_test_analysed = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    #print(x_train.shape)
    tmp = sparse.hstack((x_test_analysed,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    #print(x_test.shape)
    print("added "+str(add_cnt))
    rdforest = RandomForestClassifier(n_estimators=300)
    rdforest.fit(x_train,y_train)
    y_pred = rdforest.predict(x_test)
    gl_pred = rdforest.predict(g_test)
    predict_score_list.append(gl_pred)
    print(accuracy_score(y_test,y_pred))
    accu.append(accuracy_score(y_test,y_pred))
    conf_mat.append(confusion_matrix(y_test,y_pred))
    #classification_report.append(classification_report(y_test,y_pred))

Joy
3350
added 360
0.7149253731343284
Sadness
6746
added 473
0.6748148148148149
Anger
1366
added 231
0.6496350364963503
Disgust
2300
added 325
0.6282608695652174
Admiration
862
added 198
0.6242774566473989
Surprise
1290
added 276
0.7790697674418605
Interest
2888
added 329
0.6920415224913494
Fear
972
added 222
0.7282051282051282


In [39]:
score_list = []
for cl in df_test.columns:
    if(cl=="Text" or cl=="Analysed" or cl=='qmark' or cl=='exmark'):
        continue
    print(cl)
    score_list.append(df_test[cl].tolist())
print(len(score_list))

Joy
Sadness
Anger
Disgust
Admiration
Surprise
Interest
Fear
8


In [40]:
filter_corr = []
exmatch = 0
atleast1 = 0
md1 = 0
one_f = 0
more_f = 0
zero_f = 0
sm = 0
sdensity = 0
hammval = 0
test_len = len(predict_score_list[0])
for j in range(test_len):
    cnt=0
    for i in range(8):
        hammval+=(score_list[i][j] ^ predict_score_list[i][j])
        if(score_list[i][j]==1):
            cnt+=1
            sm+=1
    sdensity+=cnt/8
    if(cnt==0):
        zero_f+=1
    if(cnt==1):
        one_f+=1
    if(cnt>1):
        more_f+=1
    for i in range(8):
        mf = True
        if(predict_score_list[i][j]!=score_list[i][j]):
            mf=False
            break
    if(mf==True):
        exmatch+=1
        filter_corr.append(j)
    for i in range(8):
        if(predict_score_list[i][j]==score_list[i][j] and score_list[i][j]==1):
            atleast1+=1
            break
    mf = False
    for i in range(8):
        if(predict_score_list[i][j]==score_list[i][j] and score_list[i][j]==1):
            if(mf==True):
                md1+=1
                filter_corr.append(j)
                break
            mf=True
print("Label Cardinality: "+ str(sm/test_len))
print("Label Density: "+ str(sdensity/test_len))
print("Hamming Loss: "+str(hammval/(test_len*8)))
print("Exact Prediction: "+str(exmatch/test_len))
print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
print("More than one label predicted: "+str(md1/more_f))

Label Cardinality: 1.89769820971867
Label Density: 0.23721227621483376
Hamming Loss: 0.24952046035805628
Exact Prediction: 0.09207161125319693
At least one label predicted: 0.7135549872122762
More than one label predicted: 0.4213381555153707


In [41]:
tp_sum = 0
fp_sum = 0
fn_sum = 0
macro_preci = 0
macro_recall = 0
macro_f1 = 0
for i in range(len(score_list)):
    tmp = confusion_matrix(score_list[i],predict_score_list[i])
    tp_sum+=tmp[0][0]
    fp_sum+=tmp[0][1]
    fn_sum+=tmp[1][0]
    macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
    macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
    macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
    macro_preci+=macro_preci_tmp
    macro_recall+=macro_recall_tmp
    print(macro_f1)
micro_preci = tp_sum/(tp_sum+fp_sum)
micro_recall = tp_sum/(tp_sum+fn_sum)
micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
macro_preci/=8
macro_recall/=8
macro_f1/=8
print(micro_preci,micro_recall,micro_f1)
print(macro_preci,macro_recall,macro_f1)

0.7872954764196343
1.5195469977178089
2.351507459002817
3.1870337747922908
4.042151885028511
4.892846329472956
5.701202369436626
6.622195597427597
0.8116093880972338 0.8540242557883131 0.8322767809175889
0.8076941923011481 0.8509647151782322 0.8277744496784496


In [42]:
tmp = 0
for i in range(len(score_list)):
    tmp += accuracy_score(score_list[i],predict_score_list[i])
print(tmp/8)

0.7504795396419437


In [43]:
tag = []
total_len = []
train_cnt = []
test_cnt = []
accu = []
conf_mat = []
predict_score_list = []
for cl in df.columns:
    if(cl=='Text' or cl=='Analysed' or cl=='qmark' or cl=='exmark'):
        continue
    df_joy = df[df[cl]==1]
    print(cl)
    tag.append(cl)
    tmp = df[df[cl]==0].sample(len(df_joy))
    df_joy = df_joy.append(tmp).reset_index(drop=True)
    df_joy = df_joy.sample(len(df_joy)).reset_index(drop=True)
    print(len(df_joy))
    total_len.append(len(df_joy))
    #X = vectorizer.transform(df_joy['Analysed'])
    X = df_joy[['Analysed','qmark','exmark']]
    Y = df_joy[cl].tolist()
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    add_cnt = 0
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    add_cnt+=1
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    y_train.append(get_senticnet(t,cl))
    x_train_analysed = vectorizer.transform(x_train_analysed)
    x_test_analysed = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    #print(x_train.shape)
    tmp = sparse.hstack((x_test_analysed,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    #print(x_test.shape)
    print("added "+str(add_cnt))
    rdforest = SVC()
    rdforest.fit(x_train,y_train)
    y_pred = rdforest.predict(x_test)
    gl_pred = rdforest.predict(g_test)
    predict_score_list.append(gl_pred)
    print(accuracy_score(y_test,y_pred))
    accu.append(accuracy_score(y_test,y_pred))
    conf_mat.append(confusion_matrix(y_test,y_pred))
    #classification_report.append(classification_report(y_test,y_pred))

Joy
3350
added 355
0.6985074626865672
Sadness
6746
added 500
0.6585185185185185
Anger
1366
added 294
0.5656934306569343
Disgust
2300
added 356
0.6108695652173913
Admiration
862
added 193
0.5375722543352601
Surprise
1290
added 244
0.6744186046511628
Interest
2888
added 349
0.7076124567474048
Fear
972
added 198
0.6923076923076923


In [44]:
score_list = []
for cl in df_test.columns:
    if(cl=="Text" or cl=="Analysed" or cl=='qmark' or cl=='exmark'):
        continue
    print(cl)
    score_list.append(df_test[cl].tolist())
print(len(score_list))

Joy
Sadness
Anger
Disgust
Admiration
Surprise
Interest
Fear
8


In [45]:
filter_corr = []
exmatch = 0
atleast1 = 0
md1 = 0
one_f = 0
more_f = 0
zero_f = 0
sm = 0
sdensity = 0
hammval = 0
test_len = len(predict_score_list[0])
for j in range(test_len):
    cnt=0
    for i in range(8):
        hammval+=(score_list[i][j] ^ predict_score_list[i][j])
        if(score_list[i][j]==1):
            cnt+=1
            sm+=1
    sdensity+=cnt/8
    if(cnt==0):
        zero_f+=1
    if(cnt==1):
        one_f+=1
    if(cnt>1):
        more_f+=1
    for i in range(8):
        mf = True
        if(predict_score_list[i][j]!=score_list[i][j]):
            mf=False
            break
    if(mf==True):
        exmatch+=1
        filter_corr.append(j)
    for i in range(8):
        if(predict_score_list[i][j]==score_list[i][j] and score_list[i][j]==1):
            atleast1+=1
            break
    mf = False
    for i in range(8):
        if(predict_score_list[i][j]==score_list[i][j] and score_list[i][j]==1):
            if(mf==True):
                md1+=1
                filter_corr.append(j)
                break
            mf=True
print("Label Cardinality: "+ str(sm/test_len))
print("Label Density: "+ str(sdensity/test_len))
print("Hamming Loss: "+str(hammval/(test_len*8)))
print("Exact Prediction: "+str(exmatch/test_len))
print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
print("More than one label predicted: "+str(md1/more_f))

Label Cardinality: 1.89769820971867
Label Density: 0.23721227621483376
Hamming Loss: 0.27925191815856776
Exact Prediction: 0.07289002557544758
At least one label predicted: 0.6675191815856778
More than one label predicted: 0.35443037974683544


In [46]:
tp_sum = 0
fp_sum = 0
fn_sum = 0
macro_preci = 0
macro_recall = 0
macro_f1 = 0
for i in range(len(score_list)):
    tmp = confusion_matrix(score_list[i],predict_score_list[i])
    tp_sum+=tmp[0][0]
    fp_sum+=tmp[0][1]
    fn_sum+=tmp[1][0]
    macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
    macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
    macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
    macro_preci+=macro_preci_tmp
    macro_recall+=macro_recall_tmp
    print(macro_f1)
micro_preci = tp_sum/(tp_sum+fp_sum)
micro_recall = tp_sum/(tp_sum+fn_sum)
micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
macro_preci/=8
macro_recall/=8
macro_f1/=8
print(micro_preci,micro_recall,micro_f1)
print(macro_preci,macro_recall,macro_f1)

0.7568093385214008
1.4915440732561356
2.2362249243199654
3.0489339544203
3.8381866832196283
4.708968576223744
5.551777940772239
6.459102305495707
0.7904442581726739 0.8346979420225714 0.8119685717360887
0.7888613246918792 0.8317392581943708 0.8073877881869633


In [47]:
tmp = 0
for i in range(len(score_list)):
    tmp += accuracy_score(score_list[i],predict_score_list[i])
print(tmp/8)

0.7207480818414321
